In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

In [ ]:
import os
cwd = '/content/drive/MyDrive/Competitions/SIGIR/SIGIR-ecom-data-challenge-main/submission/007_stage2_mlp_rnn_02/'
os.chdir(cwd)

In [ ]:
import os
import argparse
import sys
from pathlib import Path

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

fname = 'product_mf'
checkpoint_path = Path('./checkpoints)
if not checkpoint_path.exists():
     checkpoint_path.mkdir(parents=True)

#input_path = Path('D:/SIGIR/train/')
input_path = Path('/content/drive/MyDrive/Data_Competitions/SIGIR2021/')
#input_path = Path('../input/')

In [ ]:
os.getcwd()

In [3]:
import gc
import pickle as pkl
import random
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

sys.path.append("../util")
from utils import reduce_df, save_checkpoint, seed_torch, train_epoch, val_epoch, mrr_at_k

In [193]:
num_products = 10
a = nn.Parameter(torch.Tensor(num_products)).data.normal_(0., 0.01)
a

tensor([ 0.0179, -0.0117,  0.0128,  0.0038,  0.0023, -0.0150,  0.0177,  0.0064,
        -0.0164,  0.0044])

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
torch.cuda.is_available()

In [ ]:
%%time
all_df = pd.read_csv(input_path / 'input_product_mf.csv')
print(all_df.shape)

In [ ]:
all_df

In [ ]:
all_df.dtypes

In [ ]:
reduce_df(all_df)
all_df.dtypes

In [ ]:
to_int8 = ['is_test', 'fold', 'dayofweek', 'hour', 'weekend', 'product_action_id', 'price_null', 'description_null', 'img_null']
for c in to_int8:
    all_df[c] = all_df[c].astype(np.int8)

In [ ]:
for c in all_df.columns:
    print(c, ': ', all_df[c].dtypes)

In [ ]:
all_df

## check nulls

In [ ]:
all_df.isnull().sum().sum()

In [ ]:
os.getcwd()

# Model

In [ ]:
with open(input_path / 'mappings_stage2.pkl', mode='rb') as f:
    mappings = pkl.load(f)
mappings

In [ ]:
LAGS = 19
lag_products = []

for i in range(LAGS+1):
    lag_products.append(f'product_hash_lag{i}')

lag_products

In [ ]:
LAGS_CATEGORY = 19
lag_categories = []

for i in range(LAGS_CATEGORY+1):
    lag_categories.append(f'category_lag{i}')

In [ ]:
lag_categories

In [ ]:
LAGS_URL = 19
lag_urls = []

for i in range(LAGS_URL+1):
    lag_urls.append(f'url_lag{i}')

In [ ]:
lag_urls

In [ ]:
LAGS_VECTOR = 1
lag_descriptions = []
lag_imgs = []

for k in range(4):
    for i in range(LAGS_VECTOR+1):
        lag_descriptions.append(f'description_{k}_lag{i}')
        lag_imgs.append(f'img_{k}_lag{i}')

In [ ]:
lag_descriptions

In [ ]:
lag_imgs

In [ ]:
"""import importlib
import dataset_
importlib.reload(dataset_)
from dataset_ import SigirDataset"""

In [ ]:
from dataset_ import SigirDataset

In [ ]:
list_rename = ['description_0', 'description_1', 'description_2', 'description_3', 'img_0', 'img_1', 'img_2', 'img_3']
dict_rename = {k: k + '_lag0' for k in list_rename}
dict_rename['category_hash_id'] = 'category_lag0'
dict_rename['hashed_url_id'] = 'url_lag0'

In [ ]:
dict_rename

In [ ]:
all_df = all_df.rename(columns=dict_rename)

In [ ]:
all_df.columns[:28]

In [ ]:
usecols = ['last_event_length', 'cum_pageview',
       'cum_product', 'cum_search', 'cum_event', 'lapse',
       'num_following_search', 'num_following_pageview',
       'price_bucket', 'price_null', 'description_null',
       'img_null', 'product_count',
       'product_action_id', 'first_product',
       'dayofweek', 'hour', 'weekend', 'cum_search_last', 'num_search',
       'cum_pageview_last', 'num_pageview', 'first_category', 'first_url',
       'category_hash_count', 'hashed_url_count']
usecols += ['lag_products', 'lag_categories', 'lag_urls', 'lag_descriptions', 'lag_imgs']
usecols

In [ ]:
dataset = SigirDataset(all_df, usecols, lag_targets=lag_products, lag_categories=lag_categories, lag_urls=lag_urls, lag_descriptions=lag_descriptions, lag_imgs=lag_imgs, target='next_product')

In [ ]:
dataset.__getitem__(2220)

In [ ]:
dataset.__len__()

In [ ]:
LOW_PRODUCTS = np.where(mappings[0] == -1)[0][0]
LOW_PRODUCTS

In [ ]:
NUM_CATEGORIES = all_df['category_lag0'].max() + 1
NUM_CATEGORIES

In [ ]:
NUM_URLS = all_df['url_lag0'].max() + 1
NUM_URLS

In [ ]:
from nn_ import MLP

In [ ]:
"""import importlib
import nn_
importlib.reload(nn_)
from nn_ import MLP"""

In [ ]:
os.cpu_count()

In [ ]:
TRAIN_BATCH_SIZE = 1024
VALID_BATCH_SIZE = 1024
WORKERS = os.cpu_count()
lr = 1e-3
EPOCHS = 10
GRADIENT_ACCUMULATION = 1
EMBEDDING_DIM = 64
HIDDEN_DIM =  1024
DROPOUT_RATE = 0.2
device = torch.device('cuda')
N_FOLD = 5

In [ ]:
gc.collect()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
!free -m

In [ ]:
NUM_PRODUCTS = all_df.product_hash_lag0.max() + 1
NUM_PRODUCTS

In [ ]:
len(all_df.loc[((all_df.is_test == 0) & (all_df.fold != 0) & (all_df.cum_product_r > 0)) | ((all_df.is_test==1) & (all_df.fold != 0) & (all_df.cum_product_r > 0))])

In [ ]:
len(all_df.loc[(all_df.fold != 0) & (all_df.cum_product_r > 0)])

In [ ]:
len(all_df.loc[(all_df.fold == 0) & (all_df.cum_product_r > 0) & (all_df.is_test == 0)])

In [ ]:
len(all_df[(all_df['is_test'] == 0) & (all_df['cum_product_r'] > 0) & (all_df['fold'] == 0) & (all_df['SessionId'] % 38 == 0)])

In [ ]:
start_fold = 0

In [ ]:
TRAIN_WITH_TEST = True
MULTI_GPU = False

seed = 0
seed_torch(seed)


for fold in range(start_fold, N_FOLD):
    seed_torch(seed)

    print('-'*60)
    print('FOLD %i'%(fold))

    if TRAIN_WITH_TEST:
        train = all_df.loc[(all_df.fold != fold) & (all_df.cum_product_r > 0)].copy()
    else:
        train = all_df.loc[(all_df.is_test == 0) & (all_df.fold != fold) & (all_df.cum_product_r > 0)].copy()
    valid = all_df.loc[(all_df.fold == fold) & (all_df.is_test == 0) & (all_df['cum_product_r'] > 0) & (all_df.SessionId % 38 == 0)].copy()
    print(train.shape, valid.shape)

    train_dataset = SigirDataset(train, usecols=usecols, lag_targets=lag_products, lag_categories=lag_categories, lag_urls=lag_urls, lag_descriptions=lag_descriptions, lag_imgs=lag_imgs, target='next_product')
    train_data_loader = DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        num_workers=WORKERS,
        shuffle=True,
        pin_memory=True,
    )
    valid_dataset = SigirDataset(valid, usecols=usecols, lag_targets=lag_products, lag_categories=lag_categories, lag_urls=lag_urls, lag_descriptions=lag_descriptions, lag_imgs=lag_imgs, target='next_product')
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        num_workers=WORKERS,
        shuffle=False,
        pin_memory=True,
    )

    model = MLP(NUM_PRODUCTS+1, NUM_CATEGORIES+1, NUM_URLS+1, EMBEDDING_DIM, HIDDEN_DIM, dropout_rate=DROPOUT_RATE, lag_targets=lag_products, lag_categories=lag_categories, lag_urls=lag_urls, low_targets=LOW_PRODUCTS).to(device)
    if device == 'cuda' and MULTI_GPU:
        print('use multi gpus')
        model = torch.nn.DataParallel(model) # make parallel
        cudnn.benchmark = True
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, 
                                                    pct_start=0.1, 
                                                    div_factor=1e3, 
                                                    max_lr=3e-3, 
                                                    epochs=EPOCHS, 
                                                    steps_per_epoch=int(np.ceil(len(train_data_loader)/GRADIENT_ACCUMULATION)))
    scaler = GradScaler()

    best_epoch = 0
    best_score = 0
    for epoch in range(EPOCHS):
        print(time.ctime(), 'Epoch: ', epoch, flush=True)
        train_loss = train_epoch(train_data_loader, model, optimizer, scheduler, device)
        _ = gc.collect()
        torch.cuda.empty_cache()
        val_loss, preds, labels = val_epoch(valid_data_loader, model, device) 
        preds[:, LOW_PRODUCTS] = -1e10
        score = mrr_at_k(preds, labels)

        print('Fold %d Seed %d Ep %d lr %.7f train loss %4f val loss %4f score %4f'
              % (fold, seed, epoch, 
                 optimizer.param_groups[0]["lr"],
                 np.mean(train_loss),
                 np.mean(val_loss),
                 score,
                 ), 
              flush=True)
        if score > best_score:
            best_score = score
            best_epoch = epoch
            save_checkpoint(model, epoch, optimizer, scaler, scheduler, best_loss, fold, seed, fname)
    del model, scaler, scheduler, optimizer, valid_data_loader, valid_dataset, train_data_loader, train_dataset
    _ = gc.collect()

    print('fold %d, best loss: %0.6f best epoch: %3d' % (fold, best_loss, best_epoch))